# First attempt at using CANINE

This notebook is a first test to use CANINE, the idea is to reproduce the results presented in the paper, as well as doing other experiments.

In [2]:
import os
import random as rd

import numpy as np
import pytorch_lightning as pl
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import (
    CanineForSequenceClassification,
    CanineTokenizer,
    DataCollatorWithPadding,
)

torch.set_float32_matmul_precision("medium")

In [2]:
tokenizer = CanineTokenizer.from_pretrained("google/canine-c")
model = CanineForSequenceClassification.from_pretrained("google/canine-c", num_labels=3)

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Some weights of CanineForSequenceClassification were not initialized from the model checkpoint at google/canine-c and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
dataset[
    "train"
].info.description += f"""
This dataset is a subset of the XNLI dataset. It contains {num_train_samples} samples and only the following languages: {train_language_subset}, with the following probabilities: {train_probs}.
"""

In [3]:
dataset = load_dataset("xnli", "all_languages")

Found cached dataset xnli (C:/Users/gabri/.cache/huggingface/datasets/xnli/fr/1.1.0/818164464f9c9fd15776ca8a00423b074344c3e929d00a2c1a84aa5a50c928bd)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
language_to_abbr = {
    "english": "en",
    "arabic": "ar",
    "french": "fr",
    "spanish": "es",
    "german": "de",
    "greek": "el",
    "bulgarian": "bg",
    "russian": "ru",
    "turkish": "tr",
    "chinese": "zh",
    "thai": "th",
    "vietnamese": "vi",
    "hindi": "hi",
    "urdu": "ur",
    "swahili": "sw",
}

In [14]:
num_train_samples = 50000
train_language_subset = ["en", "fr", "ar", "hi", "el", "ru", "tr", "zh"]
train_probs = [0.5, 0.3, 0.05, 0.05, 0.025, 0.025, 0.025, 0.025]
test_language_subset = ["en", "fr", "es", "bg", "th", "ur", "sw"]
test_probs = [0.5, 0.3, 0.075, 0.05, 0.025, 0.025, 0.025]

In [ ]:
def change_hypothesis_format(example):
    print(example["hypothesis"])
    example["hypothesis_dict"] = {
        k: v
        for k, v in zip(
            example["hypothesis"]["language"], example["hypothesis"]["translation"]
        )
    }
    return example


def choose_language(
    example,
    languages,
    probs,
):
    from numpy.random import choice

    lang = choice(languages, p=probs)
    example["language"] = lang
    example["choosen_premise"] = example["premise"][lang]
    example["choosen_hypothesis"] = example["hypothesis_dict"][lang]

    return example


def tokenize_dataset(example, tokenizer):
    return tokenizer(
        text=example["choosen_premise"],
        text_pair=example["choosen_hypothesis"],
        truncation=True,
    )

In [84]:
dataset["train"] = dataset["train"].map(change_hypothesis_format, num_proc=10)
dataset["validation"] = dataset["validation"].map(change_hypothesis_format, num_proc=10)
dataset["test"] = dataset["test"].map(change_hypothesis_format, num_proc=10)

Map (num_proc=10):   0%|          | 0/392702 [00:00<?, ? examples/s]

In [ ]:
dataset["train"] = dataset["train"].map(
    choose_language,
    fn_kwargs={"languages": train_language_subset, "probs": train_probs},
    batched=False,
    num_proc=10,
)
dataset["validation"] = dataset["validation"].map(
    choose_language,
    fn_kwargs={"languages": train_language_subset, "probs": train_probs},
    batched=False,
    num_proc=10,
)
dataset["test"] = dataset["test"].map(
    choose_language,
    fn_kwargs={"languages": test_language_subset, "probs": test_probs},
    batched=False,
    num_proc=10,
)

In [5]:
dataset = dataset.map(tokenize_dataset, batched=True, with_indices=False)

Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Map:   0%|          | 0/5010 [00:00<?, ? examples/s]

Map:   0%|          | 0/2490 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer, padding=True, max_length=2045, return_tensors="pt"
)

In [13]:
dataset["train"].filter(lambda x: x["input_ids"] == [])

Filter:   0%|          | 0/392702 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 0
})

In [20]:
dataset["train"].features

{'label': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [27]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [26]:
# dataset["train"] = dataset["train"].remove_columns(["premise", "hypothesis"])
dataset["train"] = dataset["train"].shuffle(seed=42).select(range(20000))
dataset["validation"] = dataset["validation"].shuffle(seed=42).select(range(2000))
dataset["test"] = dataset["test"].shuffle(seed=42).select(range(1500))

Loading cached shuffled indices for dataset at C:\Users\gabri\.cache\huggingface\datasets\xnli\fr\1.1.0\818164464f9c9fd15776ca8a00423b074344c3e929d00a2c1a84aa5a50c928bd\cache-79b8f6683ce4a7c8.arrow


In [30]:
train_dataloader = DataLoader(
    dataset["train"],
    batch_size=8,
    collate_fn=data_collator,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
)

In [31]:
batch = next(iter(train_dataloader))

In [32]:
batch

{'input_ids': tensor([[57344,    80,   111,  ...,     0,     0,     0],
        [57344,    83,   105,  ...,     0,     0,     0],
        [57344,    82,   105,  ...,     0,     0,     0],
        ...,
        [57344,    79,   117,  ...,     0,     0,     0],
        [57344,    78,   111,  ...,     0,     0,     0],
        [57344,    72,   111,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([2, 1, 0, 1, 2, 1, 2, 1])}

In [33]:
class XNLIModel(pl.LightningModule):
    def __init__(self, model, dataset, data_collator, num_workers=4, batch_size=8):
        super().__init__()
        self.model = model
        self.dataset = dataset
        self.data_collator = data_collator
        self.num_workers = num_workers
        self.batch_size = batch_size

    def forward(self, **inputs):
        return self.model(**inputs)

    def get_loss(self, batch, batch_id, step):
        outputs = self(**batch)
        loss = outputs.loss
        self.log(f"{step}_loss", loss, prog_bar=True)
        return loss

    def training_step(self, batch, batch_idx):
        return self.get_loss(batch, batch_idx, "train")

    def validation_step(self, batch, batch_idx):
        return self.get_loss(batch, batch_idx, "val")

    def test_step(self, batch, batch_idx):
        return self.get_loss(batch, batch_idx, "test")

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-5)

    def get_loader(self, split):
        return DataLoader(
            self.dataset[split],
            batch_size=self.batch_size,
            collate_fn=self.data_collator,
            shuffle=(split == "train"),
            num_workers=self.num_workers,
            pin_memory=True,
        )

    def train_dataloader(self):
        return self.get_loader("train")

    def val_dataloader(self):
        return self.get_loader("validation")

    def test_dataloader(self):
        return self.get_loader("test")

In [34]:
lit_model = XNLIModel(model, dataset, data_collator)

In [36]:
trainer = pl.Trainer(
    accelerator="gpu",
    max_epochs=3,
    accumulate_grad_batches=4,
    callbacks=[
        pl.callbacks.ModelCheckpoint(),
        pl.callbacks.EarlyStopping(monitor="val_loss", patience=3),
        pl.callbacks.LearningRateMonitor(logging_interval="step"),
    ],
    precision=16,
    fast_dev_run=True,
)

c:\Users\gabri\miniconda3\envs\pytorch-2.0\lib\site-packages\lightning_fabric\connector.py:562: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.


In [ ]:
trainer.fit(model=lit_model)